<img align="right" src="images/dhbw.png" style="width:200px"/>

# Data Mining

- Studiengang: Wirtschaftsingenieurwesen (6. Semester)
- Dozent: Tin Votan
- Datum: 21.04.2020

## 1. Python-Module importieren

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## 2. Datensätze herunterladen
Python-Skript zum Herunterladen der Datensätze, Erstellen einer Ordnerstruktur und Extrahieren der CSV-Datei.

Wann ist es sinnvoll ein Data-Scrapping-Tool in Python zu programmieren?

- Bei Änderungen der Datensätze hilft ein automatisiertes Skript die Daten unkompliziert und in der selben Ordnerstruktur herunterzuladen
- Datensätze werden auf mehreren Rechnern benötigt (Multiple-User)

In [ ]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

In [ ]:
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)

#### Datensatz bereits heruntergeladen und Ordner erstellt

In [ ]:
#fetch_housing_data()

## 3. Auslesen der Daten

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
housing = load_housing_data()
housing.head()

## 4. Einblick in die Datenstruktur

In [ ]:
housing.info()

### 4.1 Anzeigen der Kategorie `ocean_proximity`

In [ ]:
housing["ocean_proximity"].value_counts()

### 4.2 Zusammenfassung der numerischen Attribute

In [ ]:
housing.describe()

#### 1. Beispiel
25% der Distrikte in Kalifornien haben Häuser, die im Durchschnitt 18 Jahre oder jünger sind.

#### 2. Beispiel
75% der Distrikte in Kalifornien haben 1725 Einwohner oder mehr.

### 4.3 Visualierung der numerischen Attribute über ein Histogram

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

- Vertikale = Anzahl der Instanzen
- Horizontale = Wertebereich

#### 1. Beispiel
Rund 450 Distrikte in Kalifornien haben Häuser, die im Durchschnitt 18 Jahre alt sind.

#### 2. Beispiel
Rund 210 Distrikte in Kalifornien haben Häuser, die im Durchschnitt 300.000 USD wert sind.

### Vorverarbeitete Datensätze

- `housing_median_age`
- `housing_median_value`
- `median_income` 

Algorithmus könnte fälschlicherweise aus den Rohdaten lernen, dass die Preise nie höher als die Limits sind.

### Relevanz

Stellen die vorverarbeiteten Werte in `housing_median_value` eine hohe Relevanz für die Entscheidung dar?

Wenn ja, können zwei Dinge unternommen werden:
1. Die passenden Labels zu den gekappten Werten der Distrikte sammeln und aufbereiten.
2. Die Distrikte aus dem Data-Mining-Prozess entfernen, die davon betroffen sind.

## 5. Aufteilung in einen Trainingsdatensatz und einen Validierungsdatensatz

#### Data Snooping Bias
Menschen neigen dazu Datensätze automatisch auszuwerten und interessante Muster in diesen zu erkennen. Dies birgt die Gefahr, dass im Vorfeld ein Machine-Learning-Model präferiert wird.

Um dem entgegen zu wirken wird die Voraussagekraft des Models getestet.

Dazu wird der Datensatz in einen $Trainingsdatensatz$ (80%) und einen $Validierungsdatensatz$ (20%) aufgeteilt.

### 5.1 Aufteilung mit `NumPy`

In [ ]:
# to make this notebook's output identical at every run
np.random.seed(42)

In [ ]:
# For illustration only. Sklearn has train_test_split()
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train_set, test_set = split_train_test(housing, 0.2)
print(len(train_set), "train +", len(test_set), "test")

In [ ]:
import hashlib

def test_set_check(identifier, test_ratio, hash=hashlib.md5):
    return bytearray(hash(np.int64(identifier)).digest())[-1] < 256 * test_ratio

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
housing_with_id = housing.reset_index()   # adds an `index` column
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

In [ ]:
housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "id")

In [ ]:
test_set.head()

### 5.2 Aufteilung mit `Scikit-Learn`

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
test_set.head()

### 5.3 Stratified Sampling (Geschichtete Stichprobe)

Representative Darstellung / Wiedergabe der homogenen Untergruppen (= $Strata$ oder $Startum$) und der richtigen Anzahl an Instanzen von jedem Startum, z.B. Anteil Männer/Frauen an der Gesamtpopulation.

#### 5.3.1 `median_income`

In [ ]:
housing["median_income"].hist()

In [ ]:
# Divide by 1.5 to limit the number of income categories
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
# Label those above 5 as 5
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

In [ ]:
housing["income_cat"].value_counts()

In [ ]:
housing["income_cat"].hist()

| Kategorie | Wertebereich | Einkommensspanne |
| --- | --- | --- |
| 1.0 | 0.0 bis 1.5 | < 15.000 USD |
| 2.0 | 1.5 bis 3.0 | 15.000 USD bis 30.000 USD |
| 3.0 | 3.0 bis 4.5 | 30.000 USD bis 45.000 USD |
| 4.0 | 4.5 bis 6.0 | 45.000 USD bis 60.000 USD |
| 5.0 | > 6.0  | > 60.000 USD |

#### 5.3.2 `StratifiedShuffleSplit`

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

#### 5.3.3 Vergleich zwischen zufällig generierten Stichproben und geschichteten Stichproben

In [ ]:
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall": income_cat_proportions(housing),
    "Stratified": income_cat_proportions(strat_test_set),
    "Random": income_cat_proportions(test_set),
}).sort_index()

compare_props["Rand. %error"] = 100 * compare_props["Random"] / compare_props["Overall"] - 100
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

In [ ]:
compare_props

Zurücksetzen von `income_cat`

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

## 6. Erkunden und visualisieren des Datensatz

In [ ]:
housing = strat_train_set.copy()

### 6.1 Visualisierung der geografischen Daten

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")
# save_fig("bad_visualization_plot")

### 6.1.1 Visualisierung unter Berücksichtigungen der Dichte der Datenpunkte

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

### 6.1.2 Visualisierung unter Berücksichtigung von `population` und `median_house_value`

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
    s=housing["population"]/100, label="population", figsize=(10,7),
    c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False)
plt.legend()

In [ ]:
import matplotlib.image as mpimg
california_img=mpimg.imread(PROJECT_ROOT_DIR + '/images/california.png')
ax = housing.plot(kind="scatter", x="longitude", y="latitude", figsize=(10,7),
                       s=housing['population']/100, label="Population",
                       c="median_house_value", cmap=plt.get_cmap("jet"),
                       colorbar=False, alpha=0.4,
                      )
plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5,
           cmap=plt.get_cmap("jet"))
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)

prices = housing["median_house_value"]
tick_values = np.linspace(prices.min(), prices.max(), 11)
cbar = plt.colorbar()
cbar.ax.set_yticklabels(["$%dk"%(round(v/1000)) for v in tick_values], fontsize=14)
cbar.set_label('Median House Value', fontsize=16)

plt.legend(fontsize=16)
plt.show()

### 6.1.3 Korrelationen zu `median_house_value`  mithilfe einer Korrelationsmatrix

Pearson-Korrelationskoeffizient:

In [ ]:
corr_matrix = housing.corr()

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

### 6.1.4 Korrelationen zu `median_house_value`  mithilfe einer Scatter-Matrix

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))

#### Scatter-Matrix zwischen `median_income` und `median_house_value`

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)
plt.axis([0, 16, 0, 550000])

- starke (positive) Korrelation
- Preisgrenze bei 500.000 USD
- weitere horizontale Linien bei 450.000 USD, 350.000 USD und 280.000 USD

### 6.2 Experimentieren mit verschiedenen Attributkombinationen

- `total_rooms` ist schwach aussagefähig ohne `total_households` zu kennen
- interessant wäre die Anzahl der Zimmer pro Haushalt
- ebenfalls interessant ist die Anzahl der Bevölkerung pro Haushalt

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

## 7. Datenvorbereitung für die Machine-Learning-Algorithmen

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
strat_train_set.head()

In [ ]:
housing.head()

In [ ]:
housing_labels.head()

### 7.1 Data Cleaning

In [ ]:
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
sample_incomplete_rows

#### Option 1

In [ ]:
sample_incomplete_rows.dropna(subset=["total_bedrooms"])

#### Option 2

In [ ]:
sample_incomplete_rows.drop("total_bedrooms", axis=1)

#### Option 3

In [ ]:
median = housing["total_bedrooms"].median()
sample_incomplete_rows["total_bedrooms"].fillna(median, inplace=True) # option 3
sample_incomplete_rows

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

In [ ]:
housing_num = housing.drop('ocean_proximity', axis=1)

In [ ]:
imputer.fit(housing_num)

In [ ]:
imputer.statistics_

In [ ]:
housing_num.median().values

#### Leere Felder durch errechnete Medianwerte ersetzen:

In [ ]:
X = imputer.transform(housing_num)

#### Umwandlung in Pandas DataFrame:

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)